# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/e0cb0f071810107c25704c4c7c865f31749f7e121425e29e3bb131e6.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Berkeley, California, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)
    number_of_stations = len(station_locations_by_hash)
    return mplleaflet.display(), number_of_stations

map_plot, number_of_stations = leaflet_plot_stations(400,'e0cb0f071810107c25704c4c7c865f31749f7e121425e29e3bb131e6')

In [ ]:
map_plot

In [ ]:
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/e0cb0f071810107c25704c4c7c865f31749f7e121425e29e3bb131e6.csv', skiprows=1, names = ['id', 'date', 'element', 'data_value'])
df.head()

In [ ]:
import numpy as np

In [ ]:
df1 = df[['date','element','data_value']]
df_low = df1[df1['element'] == 'TMIN']
df_high = df1[df1['element'] == 'TMAX']
df_high.head()

In [ ]:
a = []
for counter, date in enumerate(np.unique(df.date)):
    if counter % 100 == 0:
        print(counter)
    low = np.min(df_low[df_low.date == date].data_value)
    high = np.max(df_high[df_high.date == date].data_value)
    a.append((date, low, high))

In [ ]:
# generate dataframe
df = pd.DataFrame(a, columns=['date','low','high'])
# add just_year and just_date columns
df['just_year'] = df.date.str[:4]
df['just_date'] = df.date.str[5:]
# remove leap year dates
df = df[df['just_date'] != '02-29']
# divide by 10
df['low'] /= 10.0
df['high'] /= 10.0
df

In [ ]:
# extract 2015 data and rename columns
df1 = pd.DataFrame(df[df.just_year == '2015'][['just_date', 'low', 'high']]).rename(columns={'just_date': 'date', 'low': '2015_low', 'high':'2015_high'})
# find lowest and highest temperatures for all but 2015 year
df1['lowest_not_2015'] = None
df1['highest_not_2015'] = None
for date in df1.date:
    df1['lowest_not_2015'][df1['date'] == date] = np.min(df[df['just_date'] == date][df['just_year'] != '2015']['low'])
    df1['highest_not_2015'][df1['date'] == date] = np.max(df[df['just_date'] == date][df['just_year'] != '2015']['high'])
df1['full_date'] = '2015-' + df1['date']
df1.head()

In [ ]:
df1['lowest'] = df1[['lowest_not_2015','2015_low']].min(axis=1)
df1['highest'] = df1[['highest_not_2015','2015_high']].max(axis=1)
df1['low_record_broken_in_2015'] = df1['2015_low'] < df1['lowest_not_2015']
df1['high_record_broken_in_2015'] = df1['2015_high'] > df1['highest_not_2015']
df1.head()

In [ ]:
low_records_broken = df1[df1.low_record_broken_in_2015 == True][['full_date', 'lowest']]
high_records_broken = df1[df1.high_record_broken_in_2015 == True][['full_date', 'highest']]

number_of_records_broken = len(low_records_broken) + len(high_records_broken)
low_records_broken

In [ ]:
# plt.figure()
observation_dates = list(map(pd.to_datetime, df1.full_date)) 
plt.plot(observation_dates, df1.highest_not_2015, '-', observation_dates, df1.lowest_not_2015, '-', linewidth=0.25)
plt.gca().fill_between(observation_dates, df1.highest,  df1.lowest, facecolor='blue', alpha=0.15)
plt.ylabel('Temperature ($^o$C)')
# plt.title('Weather temperature range for Berkeley, CA')
s1 = "2015 was an extreme year for Berkeley (CA) with 1 in 6 days setting record temperatures in a decade"
# plt.title(s1)
s2 = "{} days had record lowest and {} record\nhighest temperatures (total of {} or {} %)".format(
    len(low_records_broken), 
    len(high_records_broken), 
    number_of_records_broken,
    round(number_of_records_broken/365.0*100))
s3 = "Data collected from {} weather stations.".format(number_of_stations)
        
s4 = "Source: The National Centers for Environmental Information (NCEI).\nhttps://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt"
    
plt.text('2014-10-17', 60, s1, weight='bold', fontsize=12)

plt.text('2016-02-01', 5, '\n'.join([s2,s3]), fontsize=12)

plt.text('2016-02-01', -20, s4, fontsize=8)
# remove all the ticks (both axes), and tick labels on the Y axis
plt.tick_params(top='off', bottom='off', left='off', right='off', labelbottom='on')
plt.xticks(['2015-{}-15'.format(str(i).zfill(2)) for i in range(1,13)],
           ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
# remove the frame of the chart
for spine in plt.gca().spines.values():
    spine.set_visible(False)
    
plt.plot(high_records_broken.full_date, high_records_broken.highest, 'o',markersize=3)
plt.plot(low_records_broken.full_date, low_records_broken.lowest, 'o', markersize=3)
plt.legend(['Highest temperature for 2005-2014 span', 'Lowest temperature for 2005-2014 span', 'Record breaking high in 2015', 'Record breaking low in 2015'], bbox_to_anchor=(1.04,1), loc="upper left")

plt.show()

In [ ]:


# plt.figure()

# languages =['Python', 'SQL', 'Java', 'C++', 'JavaScript']
# pos = np.arange(len(languages))
# popularity = [56, 39, 34, 34, 29]

# # change the bar colors to be less bright blue
# bars = plt.bar(pos, popularity, align='center', linewidth=0, color='lightslategrey')
# # make one bar, the python bar, a contrasting color
# bars[0].set_color('#1F77B4')

# # soften all labels by turning grey
# plt.xticks(pos, languages, alpha=0.8)

# # TODO: remove the Y label since bars are directly labeled
# # plt.ylabel('% Popularity', alpha=0.8)
# plt.title('Top 5 Languages for Math & Data \nby % popularity on Stack Overflow', alpha=0.8)

# # remove all the ticks (both axes), and tick labels on the Y axis
# plt.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')

# # remove the frame of the chart
# for spine in plt.gca().spines.values():
#     spine.set_visible(False)
    
# # TODO: direct label each bar with Y axis values
# plt.show()